Importing Libraries

In [0]:
import requests
import json
from pyspark.sql.functions import *
from pyspark.sql.types import * 
from pyspark.sql import SparkSession

Intiating Spark Session

In [0]:
spark = SparkSession.builder.appName('API INGESTION').getOrCreate()

Getting the API JSON Data from URL

In [0]:
# JSON DATA URL
url = "https://retoolapi.dev/QWchch/data"

# JSON BODY
payload = {}
# JSON HEADER
headers = {'Content-Type': 'application/json'}

# GETTING THE DATA FROM API
response = requests.request("GET", url, headers=headers, data=payload)
print(response.text)

Saving the Input API data into Dataframe

In [0]:
Expense_df = spark.read.json(spark.sparkContext.parallelize([response.text]))
print(type(Expense_df))
Expense_df.createOrReplaceTempView("TempView_Expense")

Removing the Spaces and Special Charac and Duplicate Columns from the DataFrame

In [0]:
# Removing the spaces and special char
Expense_df_cols = [col.replace(' ','').replace(':','').lower() for col in Expense_df.columns]
# print(Expense_df_cols)

# Handling the Duplicates columns with apending indexs
duplicate_col = [idx for idx, val in enumerate(Expense_df_cols) if val in Expense_df_cols[:idx]]
for i in duplicate_col:
  Expense_df_cols[i] = Expense_df_cols[i] + '_'+ str(i)
Expense_df1 = Expense_df.toDF(*Expense_df_cols)
# print(Expense_df1)

In [0]:
Expense_df1.write.saveAsTable('`ecom-project`.silver.expense_data')